In [41]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
import time
import multiprocessing as mp
from timeit import default_timer as timer
from datetime import timedelta
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

How many cores do I have on this machine?

In [5]:
mp.cpu_count()

36

In [7]:
meta = pd.read_csv('data/arxiv_metadata_astroph.csv')
meta

/home/bkh4324/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,identifier,filename,spec,title,datestamp,created,updated,authors,categories,journal,doi,abstract,comments,date_retrieved,filename_parsed
0,oai:arXiv.org:0704.0009,704.001,physics:astro-ph,"The Spitzer c2d Survey of Large, Nearby, Inste...",2010-03-18,2007-04-02,NaN,"Paul Harvey, Bruno Merin, Tracy L. Huard, Luis...",astro-ph,"Astrophys.J.663:1149-1173,2007",10.1086/518646,We discuss the results from the combined IRA...,NaN,2020-03-07 02:10:36.300743,704.001
1,oai:arXiv.org:0704.0017,704.002,physics:astro-ph,Spectroscopic Observations of the Intermediate...,2009-06-23,2007-03-31,NaN,"Nceba Mhlahlo, David H. Buckley, Vikram S. Dhi...",astro-ph,"Mon.Not.Roy.Astron.Soc.378:211-220,2007",10.1111/j.1365-2966.2007.11762.x,Results from spectroscopic observations of t...,NaN,2020-03-07 02:10:36.300743,704.002
2,oai:arXiv.org:0704.0023,704.002,physics:astro-ph,ALMA as the ideal probe of the solar chromosphere,2009-06-23,2007-03-31,NaN,"M. A. Loukitcheva, S. K. Solanki, S. White",astro-ph,"Astrophys.Space Sci.313:197-200,2008",10.1007/s10509-007-9626-1,"The very nature of the solar chromosphere, i...",NaN,2020-03-07 02:10:36.300743,704.002
3,oai:arXiv.org:0704.0044,704.004,physics:astro-ph,Astrophysical gyrokinetics: kinetic and fluid ...,2015-05-13,2007-03-31,2009-05-09,"A. A. Schekochihin, S. C. Cowley, W. Dorland, ...",astro-ph nlin.CD physics.plasm-ph physics.spac...,"ApJS 182, 310 (2009)",10.1088/0067-0049/182/1/310,We present a theoretical framework for plasm...,NaN,2020-03-07 02:10:36.300743,704.004
4,oai:arXiv.org:0704.0048,704.005,physics:astro-ph,Inference on white dwarf binary systems using ...,2008-11-26,2007-03-31,2007-04-03,"Alexander Stroeer, John Veitch, Christian Roev...",gr-qc astro-ph,"Class.Quant.Grav.24:S541-S550,2007",10.1088/0264-9381/24/19/S17,We report on the analysis of selected single...,NaN,2020-03-07 02:10:36.300743,704.005
5,oai:arXiv.org:0704.0059,704.006,physics:astro-ph,The Mass and Radius of the Unseen M-Dwarf Comp...,2009-06-23,2007-03-31,2007-04-09,"T. G. Beatty, J. M. Fernandez, D. W. Latham, G...",astro-ph,"Astrophys.J.663:573-582,2007",10.1086/518413,We derive masses and radii for both componen...,NaN,2020-03-07 02:10:36.300743,704.006
6,oai:arXiv.org:0704.0080,704.008,physics:astro-ph,Shaping the Globular Cluster Mass Function by ...,2010-11-11,2007-04-01,2008-06-11,"Dean E. McLaughlin, S. Michael Fall",astro-ph,"Astrophys.J.679:1272-1287,2008",10.1086/533485,We show that the globular cluster mass funct...,NaN,2020-03-07 02:10:36.300743,704.008
7,oai:arXiv.org:0704.0094,704.009,physics:astro-ph,Timing and Lensing of the Colliding Bullet Clu...,2007-05-23,2007-04-02,NaN,HongSheng Zhao,astro-ph,NaN,NaN,We present semi-analytical constraint on the...,NaN,2020-03-07 02:10:36.300743,704.009
8,oai:arXiv.org:0704.0128,704.013,physics:astro-ph,An online repository of Swift/XRT light curves...,2009-11-13,2007-04-02,2007-04-19,"P. A. Evans, A. P. Beardmore, K. L. Page, L. G...",astro-ph,NaN,10.1051/0004-6361:20077530,Context. Swift data are revolutionising our ...,NaN,2020-03-07 02:10:36.300743,704.013
9,oai:arXiv.org:0704.0133,704.013,physics:astro-ph,PAH emission and star formation in the host of...,2009-11-13,2007-04-02,NaN,"D. Lutz, E. Sturm, L. J. Tacconi, E. Valiante,...",astro-ph,NaN,10.1086/518537,We report the first detection of the 6.2micr...,NaN,2020-03-07 02:10:36.300743,704.013


Collect data (abstracts) that we will train doc2vec on:

In [8]:
data = [abstract.replace('\n', ' ') for abstract in meta['abstract']]
len(data)

267794

Tokenize each abstract. No stopwords are removed.

In [20]:
%%timeit
train_corpus = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i,_d in enumerate(data)]

4min 14s ± 403 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
train_corpus[0]

TaggedDocument(words=['we', 'discuss', 'the', 'results', 'from', 'the', 'combined', 'irac', 'and', 'mips', 'c2d', 'spitzer', 'legacy', 'observations', 'of', 'the', 'serpens', 'star-forming', 'region', '.', 'in', 'particular', 'we', 'present', 'a', 'set', 'of', 'criteria', 'for', 'isolating', 'bona', 'fide', 'young', 'stellar', 'objects', ',', 'yso', "'s", ',', 'from', 'the', 'extensive', 'background', 'contamination', 'by', 'extra-galactic', 'objects', '.', 'we', 'then', 'discuss', 'the', 'properties', 'of', 'the', 'resulting', 'high', 'confidence', 'set', 'of', 'yso', "'s", '.', 'we', 'find', '235', 'such', 'objects', 'in', 'the', '0.85', 'deg^2', 'field', 'that', 'was', 'covered', 'with', 'both', 'irac', 'and', 'mips', '.', 'an', 'additional', 'set', 'of', '51', 'lower', 'confidence', 'yso', "'s", 'outside', 'this', 'area', 'is', 'identified', 'from', 'the', 'mips', 'data', 'combined', 'with', '2mass', 'photometry', '.', 'we', 'describe', 'two', 'sets', 'of', 'results', ',', 'color-c

Set up model parameters.

dm = 0 means "distributed bag of words" (PV-DBOW)

dm = 1 means "distributed memory" (PV-DM)

In [47]:
model = Doc2Vec(vector_size=300,
                alpha=0.025,
                min_alpha=0.001, 
                min_count=1,
                workers=mp.cpu_count(),
                dm=1,
                epochs=40)

model.build_vocab(train_corpus)

INFO:gensim.models.doc2vec:collecting all words and their counts
INFO:gensim.models.doc2vec:PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #10000, processed 1944145 words (6991563/s), 50620 word types, 10000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #20000, processed 3946520 words (6949936/s), 77936 word types, 20000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #30000, processed 5978508 words (6592293/s), 101136 word types, 30000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #40000, processed 8080110 words (6581085/s), 123023 word types, 40000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #50000, processed 10169046 words (6455802/s), 143299 word types, 50000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #60000, processed 12286117 words (6496236/s), 162642 word types, 60000 tags
INFO:gensim.models.doc2vec:PROGRESS: at example #70000, processed 14469579 words (6429567/s), 18180

Train model for 100 epochs. 

In [48]:
start = timer()
max_epochs = 100

model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

model.save('d2v.model')
print('Model Saved')
end = timer()
print ("Execution time HH:MM:SS:",timedelta(seconds=end-start))

INFO:gensim.models.base_any2vec:training model with 36 workers on 452976 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 0.02% examples, 4617 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 0.69% examples, 84173 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 1.35% examples, 117725 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 2.04% examples, 134253 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 2.74% examples, 144274 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 3.43% examples, 152807 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 4.07% examples, 162571 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 4.52% example

INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 32.27% examples, 169512 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 32.71% examples, 169802 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 33.17% examples, 169724 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 33.53% examples, 169566 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 33.94% examples, 169488 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 34.42% examples, 169959 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 34.78% examples, 169821 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 35.09% examples, 169272 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 35.58% examples, 169819 words/s, in_qsize

INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 60.59% examples, 168315 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 60.93% examples, 168191 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 61.34% examples, 168261 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 61.73% examples, 168328 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 62.07% examples, 168092 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 62.48% examples, 168166 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 62.89% examples, 168184 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 63.22% examples, 168081 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 63.64% examples, 168224 words/s, in_qsize

INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 92.10% examples, 166169 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 92.56% examples, 166175 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 93.04% examples, 166177 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 93.46% examples, 166055 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 93.89% examples, 165993 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 94.35% examples, 165969 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 94.73% examples, 165849 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 95.28% examples, 165903 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 95.78% examples, 165960 words/s, in_qsize

INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 12.59% examples, 163190 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 13.02% examples, 163749 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 13.36% examples, 163170 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 13.77% examples, 163480 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 14.18% examples, 163377 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 14.58% examples, 163436 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 15.01% examples, 163874 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 15.46% examples, 164093 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 15.90% examples, 164161 words/s, in_qsize

...

INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 12.15% examples, 173536 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 12.60% examples, 173717 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 13.04% examples, 173506 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 13.43% examples, 173223 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 13.83% examples, 173001 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 14.24% examples, 172995 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 14.73% examples, 174094 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 15.15% examples, 174089 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 15.69% examples, 175052 words/s,

INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 43.06% examples, 175263 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 43.47% examples, 175009 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 43.89% examples, 175125 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 44.30% examples, 175160 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 44.67% examples, 175075 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 45.07% examples, 174990 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 45.48% examples, 175090 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 45.93% examples, 175379 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 46.38% examples, 175497 words/s,

INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 72.65% examples, 174399 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 73.10% examples, 174313 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 73.58% examples, 174308 words/s, in_qsize 72, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 74.06% examples, 174348 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 74.44% examples, 174062 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 74.93% examples, 174061 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 75.38% examples, 173973 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 75.89% examples, 173948 words/s, in_qsize 71, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 40 - PROGRESS: at 76.28% examples, 173793 words/s,

INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 15 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 14 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 13 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 12 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 11 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 10 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 9 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 8 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 7 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 6 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 5 more threads
INFO

FileNotFoundError: [Errno 2] No such file or directory: '../results/2020_04_03_abstracts_doc2vec/d2v.model.trainables.syn1neg.npy'

In [49]:
model.save('d2v.model')

INFO:gensim.utils:saving Doc2Vec object under d2v.model, separately None
INFO:gensim.utils:storing np array 'syn1neg' to d2v.model.trainables.syn1neg.npy
INFO:gensim.utils:storing np array 'vectors' to d2v.model.wv.vectors.npy
INFO:gensim.utils:storing np array 'vectors_docs' to d2v.model.docvecs.vectors_docs.npy
INFO:gensim.utils:saved d2v.model
